# Test Notebook

Hello! This notebook is a test to see if all the code migration has worked properly. It will also test integration of notebooks into a mkdocs file. Let us observe!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import binned_statistic_dd as hist

# Move into the source directory for this notebook to work properly
# Probably want a better way of doing this.
import os
import importlib
os.chdir('../src/')

# Import whatever we need
import disruptivity as dis
import vis.disruptivity_vis as dis_vis
import vis.probability_vis as prob_vis
from vis.plot_helpers import plot_subplot as plot
import data_loader

importlib.reload(dis)
importlib.reload(prob_vis)
load_disruptions_mat = data_loader.load_disruptions_mat

## CMOD Data

In [ ]:
cmod_df, cmod_indices = load_disruptions_mat('../data/CMod_disruption_warning_db.mat')
n_shots = np.unique(cmod_df.shot).shape[0]
n_shots_no_disrupt = np.unique(cmod_df.shot[cmod_indices['indices_no_disrupt']]).shape[0]
n_shots_disrupt = np.unique(cmod_df.shot[cmod_indices['indices_disrupt']]).shape[0]
assert n_shots_disrupt+n_shots_no_disrupt == n_shots, \
    'Number of disrupts plus number of non disruptions does not equal the total shot number'
print(f'Total Shot Number: {n_shots}, Non-Disrupted Shots: {n_shots_no_disrupt}, Disrupted Shots: {n_shots_disrupt}')

# Hand Tuned Parameters from Nadia
LI_range = [ 0.75, 2.25] 
BETAN_range = [0, 1.5]
KAPPA_range = [0.8, 2.0]
GAP_in_range = [0.05, 0.25]
GAP_out_range = [0, 0.2]
Q95_range = [2, 7.5]
I_P_range = [-1.5e6, 1.5e6]

# These dictionaries will automatically be generated by the structure that will surround this code
# This is yet to be done, but this is what I was planning to do anyway.
entry_dict_1D = {
   'kappa':{
        'range':KAPPA_range
    },
}

entry_dict_IP = {
    'ip':{
        'range':I_P_range
    },
}

entry_dict_2D = {
    'kappa':{
        'range':KAPPA_range
    },
    'ip':{
        'range':I_P_range
    },
 }

Some somewhat unecessary index manipulation as proof of concept of how this would work for a new type of disruption.

In [ ]:
'''
So my goal with this block of code is to find all the portions of flat top disrupted shots 
that are in flat tops. Should be simple enough.
'''

# Step 1: Find all indices of flat top disrupted shots.
flattop_disrupt_shot_number = cmod_df.shot[cmod_indices['indices_disrupt_time_in_flattop']]
flattop_disrupt_shot_indices = cmod_df[np.isin(cmod_df.shot,flattop_disrupt_shot_number)].index


# Step 2: Find where the flattop indices and flattop disruption indicies intersect. Done!
indices_flattop_disrupt = np.intersect1d(flattop_disrupt_shot_indices, cmod_indices['indices_flattop'])

# Now the flattop non disrupted data.
indices_flattop_no_disrupt = np.intersect1d(cmod_indices['indices_flattop'], cmod_indices['indices_no_disrupt'])

# Get the possible warning times
disrupt_warning_time_ms = 50
disrupt_warning_window_ms = 10
possible_warning_times = np.abs(cmod_df.time_until_disrupt-disrupt_warning_time_ms) < disrupt_warning_time_ms

# Make sure they are part of our disruptions of interest
indices_n_disrupt = np.array(
    np.intersect1d(
        indices_flattop_disrupt,
        cmod_df.loc[possible_warning_times].index
    ))

# All other flattop data points
indices_n_total = np.array(indices_flattop_no_disrupt)

# Compute Kappa disruptivity
disr, err, bins = dis.compute_disruptivity(cmod_df, entry_dict_1D, indices_n_disrupt, indices_n_total)
args = [disr, err, *bins]
fig, ax = plot("cmod_kappa_disruptivity.png", dis_vis.subplot_disruptivity1d, args)
ax.set_xlabel("$\kappa$")

# Compute IP disruptivity
disr, err, bins = dis.compute_disruptivity(cmod_df, entry_dict_IP, indices_n_disrupt, indices_n_total)
args = [disr, err, *bins]
fig, ax = plot("cmod_IP_disruptivity.png", dis_vis.subplot_disruptivity1d, args)
ax.set_xlabel("$I_p$ (A)")

In [ ]:
# Get the Ip histograms
disr, err, bins = dis.compute_disruptivity(cmod_df, entry_dict_1D, indices_n_disrupt, indices_n_total, dt=0.002)
args = [disr, err, *bins]
fig, ax = plot("cmod_kappa_const_dt_disruptivity.png", dis_vis.subplot_disruptivity1d, args)
ax.set_xlabel("$I_p$ (A)")

# Get the Ip histograms
disr, err, bins = dis.compute_disruptivity(cmod_df, entry_dict_IP, indices_n_disrupt, indices_n_total, dt=0.002)
args = [disr, err, *bins]
fig, ax = plot("cmod_ip_const_dt_disruptivity.png", dis_vis.subplot_disruptivity1d, args)
ax.set_xlabel("$I_p$ (A)")

In [ ]:
# Get the 2D histograms
disr, err, bins = dis.compute_disruptivity(cmod_df, entry_dict_2D, indices_n_disrupt, indices_n_total)
args = [disr, err, *bins]
fig,ax = plot("cmod_kappa_ip_disruptivity.png", dis_vis.subplot_disruptivity2d, args)
ax.set_ylabel("$I_p$ (A)")
ax.set_xlabel("$\kappa$")

# Get the 2D histograms
disr, err, bins = dis.compute_disruptivity(cmod_df, entry_dict_2D, indices_n_disrupt, indices_n_total, dt=0.002)
args = [disr, err, *bins]
fig, ax = plot("cmod_kappa_ip_const_dt_disruptivity.png", dis_vis.subplot_disruptivity2d, args)
ax.set_ylabel("$I_p$")
ax.set_xlabel("$\kappa$")

## DIII-D Data

In [ ]:
d3d_df, d3d_indices = load_disruptions_mat('../data/d3d-db-220420.mat')
n_shots = np.unique(cmod_df.shot).shape[0]
n_shots_no_disrupt = np.unique(cmod_df.shot[cmod_indices['indices_no_disrupt']]).shape[0]
n_shots_disrupt = np.unique(cmod_df.shot[cmod_indices['indices_disrupt']]).shape[0]
assert n_shots_disrupt+n_shots_no_disrupt == n_shots, \
    'Number of disrupts plus number of non disruptions does not equal the total shot number'
print(f'Total Shot Number: {n_shots}, Non-Disrupted Shots: {n_shots_no_disrupt}, Disrupted Shots: {n_shots_disrupt}')

In [ ]:
'''
So my goal with this block of code is to find all the portions of flat top disrupted shots 
that are in flat tops. Should be simple enough.
'''

# Step 1: Find all indices of flat top disrupted shots.
flattop_disrupt_shot_number = d3d_df.shot[d3d_indices['indices_disrupt_time_in_flattop']]
flattop_disrupt_shot_indices = d3d_df[np.isin(d3d_df.shot,flattop_disrupt_shot_number)].index


# Step 2: Find where the flattop indices and flattop disruption indicies intersect. Done!
indices_flattop_disrupt = np.intersect1d(flattop_disrupt_shot_indices, d3d_indices['indices_flattop'])

# Now the flattop non disrupted data.
indices_flattop_no_disrupt = np.intersect1d(d3d_indices['indices_flattop'], d3d_indices['indices_no_disrupt'])

# Get the possible warning times
disrupt_warning_time_ms = 50
disrupt_warning_window_ms = 10
possible_warning_times = np.abs(d3d_df.time_until_disrupt-disrupt_warning_time_ms) < disrupt_warning_time_ms

# Make sure they are part of our disruptions of interest
indices_n_disrupt = np.array(
    np.intersect1d(
        indices_flattop_disrupt,
        d3d_df.loc[possible_warning_times].index
    ))

# All other flattop data points
indices_n_total = np.array(indices_flattop_no_disrupt)

# Get the Kappa histograms
disr, err, bins = dis.compute_disruptivity(d3d_df, entry_dict_1D, indices_n_disrupt, indices_n_total)
args = [disr, err, *bins]
fig, ax = plot("d3d_kappa_disruptivity.png", dis_vis.subplot_disruptivity1d, args)
ax.set_xlabel("$\kappa$")

# Get the Ip histograms
disr, err, bins = dis.compute_disruptivity(d3d_df, entry_dict_IP, indices_n_disrupt, indices_n_total)
args = [disr, err, *bins]
fig, ax = plot("d3d_ip_disruptivity.png", dis_vis.subplot_disruptivity1d, args)
ax.set_xlabel("$I_p$ (A)")

In [ ]:
# Get the Ip histograms
disr, err, bins = dis.compute_disruptivity(d3d_df, entry_dict_IP, indices_n_disrupt, indices_n_total, dt=0.001)
args = [disr, err, *bins]
fig, ax = plot("d3d_ip_const_dt_disruptivity.png", dis_vis.subplot_disruptivity1d, args)
ax.set_xlabel("$I_p$ (A)")

In [ ]:
# Get the 2D histograms
disr, err, bins = dis.compute_disruptivity(d3d_df, entry_dict_2D, indices_n_disrupt, indices_n_total)
args = [disr, err, *bins]
fig, ax = plot("d3d_kappa_ip_disruptivity.png", dis_vis.subplot_disruptivity2d, args)
ax.set_ylabel("$I_p$")
ax.set_xlabel("$\kappa$")

In [ ]:
# Get the 2D histograms
disr, err, bins = dis.compute_disruptivity(d3d_df, entry_dict_2D, indices_n_disrupt, indices_n_total, dt=0.001)
args = [disr, err, *bins]
fig, ax = plot("d3d_kappa_ip_disruptivity.png", dis_vis.subplot_disruptivity2d, args)
ax.set_ylabel("$I_p$")
ax.set_xlabel("$\kappa$")